<a href="https://colab.research.google.com/github/askaration/biodiversity_taxonomy/blob/main/occurrences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing necessary libraries that are not available by default
!pip install geopy
!pip install geocoder

# Importing libraries
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time
import geocoder


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = "/content/drive/MyDrive/Spring 2024/Independent Study/Week 12/occurrences.csv"

In [ ]:
data = pd.read_csv(file_path, encoding="latin1", low_memory=False)
data.columns

Index(['id', 'institutionCode', 'collectionID', 'basisOfRecord',
       'occurrenceID', 'catalogNumber', 'otherCatalogNumbers',
       'higherClassification', 'kingdom', 'phylum', 'class', 'order', 'family',
       'scientificName', 'taxonID', 'scientificNameAuthorship', 'genus',
       'subgenus', 'specificEpithet', 'verbatimTaxonRank',
       'infraspecificEpithet', 'taxonRank', 'identifiedBy', 'dateIdentified',
       'identificationRemarks', 'recordedBy', 'recordNumber', 'eventDate',
       'year', 'verbatimEventDate', 'occurrenceRemarks', 'habitat',
       'informationWithheld', 'dynamicProperties', 'associatedOccurrences',
       'associatedTaxa', 'reproductiveCondition', 'lifeStage', 'sex',
       'country', 'stateProvince', 'county', 'municipality', 'locality',
       'decimalLatitude', 'decimalLongitude', 'geodeticDatum',
       'coordinateUncertaintyInMeters', 'georeferenceProtocol',
       'georeferenceSources', 'georeferenceVerificationStatus',
       'georeferenceRemarks',

In [ ]:
results = []


for column in data.columns:
    empty_cells = data[column].isnull().sum()
    empty_percentage = round((empty_cells/data.shape[0]) * 100, 2)
    # if empty_percentage >= 30:
    (print(f"Column {column} has {empty_cells} empty cells, which makes {empty_percentage}% of the column. \n"))

    result_dict = {
        "column": column,
        "empty cells": empty_cells,
        "percentage": empty_percentage
    }

        # Append the dictionary to the "results" list
    results.append(result_dict)

# Create a DataFrame from the "results" list
emptyCellResult = pd.DataFrame(results)

# Save the DataFrame to a CSV file
# emptyCellResult.to_csv("empty_cells_analysis.csv", index=False)

Column id has 0 empty cells, which makes 0.0% of the column. 

Column institutionCode has 0 empty cells, which makes 0.0% of the column. 

Column collectionID has 0 empty cells, which makes 0.0% of the column. 

Column basisOfRecord has 0 empty cells, which makes 0.0% of the column. 

Column occurrenceID has 0 empty cells, which makes 0.0% of the column. 

Column catalogNumber has 32 empty cells, which makes 0.02% of the column. 

Column otherCatalogNumbers has 44624 empty cells, which makes 33.91% of the column. 

Column higherClassification has 768 empty cells, which makes 0.58% of the column. 

Column kingdom has 768 empty cells, which makes 0.58% of the column. 

Column phylum has 771 empty cells, which makes 0.59% of the column. 

Column class has 106956 empty cells, which makes 81.27% of the column. 

Column order has 771 empty cells, which makes 0.59% of the column. 

Column family has 769 empty cells, which makes 0.58% of the column. 

Column scientificName has 86 empty cells, 

In [ ]:
darwin_core_terms = ["occurrenceID", "catalogNumber", "recordNumber", "recordedBy",
    "individualCount", "organismQuantity", "organismQuantityType", "sex",
    "lifeStage", "reproductiveCondition", "behavior", "establishmentMeans",
    "occurrenceStatus", "preparations", "disposition", "associatedMedia",
    "associatedReferences", "associatedSequences", "associatedTaxa",
    "otherCatalogNumbers", "occurrenceRemarks", "organismID", "organismName",
    "organismScope", "associatedOccurrences", "associatedOrganisms",
    "previousIdentifications", "eventDate", "eventTime", "startDayOfYear",
    "endDayOfYear", "year", "month", "day", "verbatimEventDate", "habitat",
    "samplingEffort", "samplingProtocol", "fieldNumber", "eventID",
    "parentEventID", "fieldNotes", "eventRemarks", "locationID",
    "higherGeographyID", "higherGeography", "continent", "waterBody",
    "islandGroup", "island", "country", "countryCode", "stateProvince",
    "county", "municipality", "locality", "verbatimLocality",
    "verbatimElevation", "verbatimDepth", "minimumDistanceAboveSurfaceInMeters",
    "maximumDistanceAboveSurfaceInMeters", "locationAccordingTo",
    "locationRemarks", "decimalLatitude", "decimalLongitude", "geodeticDatum",
    "coordinateUncertaintyInMeters", "coordinatePrecision",
    "pointRadiusSpatialFit", "verbatimCoordinates", "verbatimLatitude",
    "verbatimLongitude", "verbatimCoordinateSystem", "verbatimSRS",
    "footprintWKT", "footprintSRS", "footprintSpatialFit", "georeferencedBy",
    "georeferencedDate", "georeferenceProtocol", "georeferenceSources",
    "georeferenceVerificationStatus", "georeferenceRemarks",
    "geologicalContextID", "earliestEonOrLowestEonothem",
    "latestEonOrHighestEonothem", "earliestEraOrLowestErathem",
    "latestEraOrHighestErathem", "earliestPeriodOrLowestSystem",
    "latestPeriodOrHighestSystem", "earliestEpochOrLowestSeries",
    "latestEpochOrHighestSeries", "earliestAgeOrLowestStage",
    "latestAgeOrHighestStage", "lowestBiostratigraphicZone",
    "highestBiostratigraphicZone", "lithostratigraphicTerms", "group",
    "formation", "member", "bed", "identificationID", "identificationQualifier",
    "typeStatus", "identifiedBy", "dateIdentified", "identificationReferences",
    "identificationVerificationStatus", "identificationRemarks", "taxonID",
    "scientificName", "acceptedNameUsage", "parentNameUsage",
    "originalNameUsage", "nameAccordingTo", "namePublishedIn",
    "namePublishedInYear", "higherClassification", "kingdom", "phylum", "class",
    "order", "family", "genus", "subgenus", "specificEpithet",
    "infraspecificEpithet", "taxonRank", "verbatimTaxonRank",
    "scientificNameAuthorship", "vernacularName", "nomenclaturalCode",
    "taxonomicStatus", "nomenclaturalStatus", "taxonRemarks", "measurementID",
    "measurementType", "measurementValue", "measurementAccuracy",
    "measurementUnit", "measurementDeterminedBy", "measurementDeterminedDate",
    "measurementMethod", "measurementRemarks", "resourceRelationshipID",
    "resourceID", "relatedResourceID", "relationshipOfResource",
    "relationshipAccordingTo", "relationshipEstablishedDate",
    "relationshipRemarks"
]

# Species and countrycode do not exist

# matching terms equals column, for every column in the dataframe if the column exists in the darwin core terms

matching_terms = [column for column in data.columns if column in darwin_core_terms]
matching_terms = pd.Series(matching_terms)
print(f"There are {matching_terms.shape[0]} columns that match the Darwin Core Occurrence Class out of {data.shape[1]} columns in the dataset.")
# matching_terms.to_csv("Matching DC Terms.csv", index=False)

There are 46 columns that match the Darwin Core Occurrence Class out of 59 columns in the dataset.


In [ ]:
county_path = '/content/drive/MyDrive/Spring 2024/Independent Study/Week 12/counties_curated.csv'

counties = pd.read_csv(county_path, encoding="latin1", low_memory=False)
print(counties.columns)

import pandas as pd
import numpy as np

# Assuming 'data' and 'counties' are your DataFrames already loaded
location = data[['county', 'stateProvince', 'decimalLongitude', 'decimalLatitude']]

# Merge the DataFrames on 'stateProvince' and 'county'
merged_df = pd.merge(location, counties[['county', 'stateProvince', 'decimalLongitude', 'decimalLatitude']],
                     on=['county', 'stateProvince'], suffixes=('', '_from_counties'), how='left')

# Function to update coordinates if they are NaN
def update_coordinates(row):
    if pd.isna(row['decimalLongitude']):
        row['decimalLongitude'] = row['decimalLongitude_from_counties']
    if pd.isna(row['decimalLatitude']):
        row['decimalLatitude'] = row['decimalLatitude_from_counties']
    return row

# Apply the function to each row
updated_location = merged_df.apply(update_coordinates, axis=1)

# Drop the extra columns from counties
updated_location.drop(columns=['decimalLongitude_from_counties', 'decimalLatitude_from_counties'], inplace=True)

# Check results
# print(updated_location.head())

new_data = data.drop(['decimalLongitude', 'decimalLatitude', 'county', 'stateProvince'], axis=1)
data = pd.concat([new_data, updated_location], axis=1)


# Save the modified dataset
data.to_csv("newOccurrences.csv", index=False)


# data.head()


emptyLongitude = data['decimalLongitude'].isnull().sum()
longPercent = round((emptyLongitude/data.shape[0])*100, 2)
emptyLatitude = data['decimalLatitude'].isnull().sum()
latPercent = round((emptyLatitude/data.shape[0])*100, 2)


print(f'After imputation and finding some of the missing coordinates, we went from 94.78% missing conrdinates into {longPercent}%')

Index(['stateProvince', 'county', 'decimalLongitude', 'decimalLatitude'], dtype='object')
After imputation and finding some of the missing coordinates, we went from 94.78% missing conrdinates into 28.55%
